#### Student Name: Mai Ngo
#### Course Name and Number: CSC 575 Intelligent Information Retrieval - SEC 801
#### Assignment 3 - Vector-space Retrieval and Evaluation
#### Date: 2/4/2024

### Step 1: Load Inverted Index and compute Doc Vector Length.

In [1]:
import csv
import math
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from itertools import islice

#### Read the term index file and populate the inverted index.

In [2]:
#Total number of docs in the corpus.
N = 1033

#Dictionary for inverted index {term as key | tuple(idf, list of postings) as value}.
#idf: of a term in the entire document corpus. JUST THE IDF!!! -- 
#List of postings: element is individual dictionary {doc ID as keys | raw term frequency as value}.
term_invIndex = {} 

#Dictionary {doc ID as key | vector length of it as value}.
docID_lenDict = {} 

#Dictionary {term ID as key | term as value}
termID_termDict = {}

#Each word (string) to its term ID and document frequency - count of docs where the term appears.
term_indexFile = open('medline_term_index.csv', 'r', encoding='utf-8')
reader = csv.reader(term_indexFile, delimiter='\t')

#Extract individual columns.
for line in reader:
    term = line[0]              #term string
    termID = line[1]            #termID
    df = int(line[2])           #document frequency
    idf = math.log10(N/df)      #idf
    
    #For each term, values as tuples (idf, empty dictionary)
    term_invIndex[term] = (idf, dict())
    #For each term ID, value as corresponding term 
    termID_termDict[termID] = term 
term_indexFile.close()

#### Read the inverted index file and add postings lists in term_invIndex dicitionary.

In [3]:
#Each term ID to positing lists (doc ID and count of term within).
inv_indexFile = open('medline_inverted_index.csv', 'r')
reader = csv.reader(inv_indexFile, delimiter='\t')
for line in reader:
    termID = line[0]
    idx = 1
    while idx < (len(line)-1):
        docID = line[idx] #A string
        #Raw count of the term in this document, convert to integer. 
        termFreq = int(line[idx+1]) 
        
        #Nested dictionary for each individual posting element {doc ID as key | raw termFreq as value}.
        #postingDict is generated by using termID in termID_termDict, to get corresponding key termDict. 
        #Further, using that termDict to retrieve value from term_invIndex. 
        #Since that value was initialized as a tuple with 2nd element as a dictionary. 
        #Each posting will be deemed as individual postingDict. 
        #Keep creating till end of each line/row.
        postingDict = (term_invIndex[termID_termDict[termID]])[1]
        postingDict[docID] = termFreq  
        
        #Accumulate the component vector length for the document.
        #Compute Sqrt tf_idf for each term in current document 
        tf_idf = termFreq * (term_invIndex[termID_termDict[termID]])[0] 
        tf_idfSq = math.pow(tf_idf, 2.0)
        if docID in docID_lenDict:
            docID_lenDict[docID] += tf_idfSq
        else:
            docID_lenDict[docID] = tf_idfSq
        
        #Increase for next postings within a row.
        idx += 2
inv_indexFile.close()

#Normalized vector represents each single document in the corpus.
for docID in docID_lenDict.keys():
    #Take Sqrt of of the accumulated Sqrt of tf_idf
    val = docID_lenDict[docID]
    docID_lenDict[docID] = math.sqrt(val)

    
print ('Total # terms: %d' % len(term_invIndex))
for term in ['pentobarbit', 'defici', 'treatment']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(term_invIndex[term][1]), term_invIndex[term][0]))

print ('\nTotal # documents: %d' % len(docID_lenDict))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, docID_lenDict[docID]))

Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
 - Entry for 'defici': df=39, idf=1.4230357144931214
 - Entry for 'treatment': df=172, idf=0.7785718746120717

Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


### Step 2: Store queries data.

In [4]:
#List of queries. Each element is a tuple (query ID, nested dictionary {term as key | term frequency as value}.
#Term frequency: Total occurrence of a term in query corpus.
queriesList = []

#30 queries. query ID followed by the query as strings.
queryFile = open('medline.query', 'r', encoding='utf-8')#'iso-8859-1'
porter = nltk.PorterStemmer()

for line in queryFile:
    matchObj = re.match(r'^(\d+)\s+(.*)', line)
    if not matchObj:
        print ("ERROR with line -- %s" % line)
    else:
        queryID = matchObj.group(1) #queryID
        text = matchObj.group(2)    #query string (ignore sentences)

        #Process text string
        tokens = word_tokenize(text.lower())
        terms = [porter.stem(w) for w in tokens if w not in stopwords.words('english') and len(w) > 1] # (**)
        
        #Dictionary term frequency. For each query, {term as key | frequency as value}. 
        fdist = nltk.FreqDist(terms)
        
        #Each element as tuple (queryID, term freq dictionary).
        queriesList.append((queryID, dict(fdist)))
queryFile.close()

print ('Total # queries: %d' % len(queriesList))
for queryID in [1, 21]:
    print (' - Query %s: %s' % (queriesList[queryID][0], queriesList[queryID][1]))

Total # queries: 30
 - Query 2: {'relationship': 1, 'blood': 1, 'cerebrospin': 1, 'fluid': 1, 'oxygen': 1, 'concentr': 1, 'partial': 1, 'pressur': 1, 'method': 1, 'interest': 1, 'polarographi': 1}
 - Query 22: {'mycoplasma': 1, 'infect': 1, 'presenc': 1, 'embryo': 1, 'fetu': 1, 'newborn': 1, 'infant': 1, 'anim': 1, 'pregnanc': 1, 'gynecolog': 1, 'diseas': 1, 'relat': 1, 'chromosom': 2, 'abnorm': 1}


### Step 3: Inverted-Index Retrieval Algorithm

#### Compute TF-IDF for the queries -- using IDF with respect to document corpus. 

In [5]:
#Dictionary to store tf-idf of the queries. queryID as keys | nested dictionary term_tf_idf as values.
#Respective value are a dictionary with term as keys and its tf-idf as value.
tf_idfQueries = {}

totalQueries = len(queriesList)

#Nested loops to retrieve term frequency with respect to query corpus. 
#1st Iteration: Extract term dictionary for each query. 
#2nd Iteration: Extract terms (keys) from each term dictionary.
#For each term, if found in other queries, add 1 to the accumulator sequence. 
#Sum to get total occurences in the query corpus.  
termFrequencyDict = {term: sum(1 for qID, otherTerms in queriesList if term in otherTerms) for _, termDict in queriesList for term in termDict.keys()}

#Iterate over each query in queriesList.
for queryID, termDict in queriesList:
    
    tf_idfQueries[queryID] = {}

    #Iterate over each term in current query.
    for term, raw_tf in termDict.items():
        #Check if the term exists in the document corpus vocabulary (term_invIndex)
        if term in term_invIndex:
            
            #Retrieve the respective IDF. First element of the tuple.
            termIDF = term_invIndex[term][0]

            #TF-IDF = raw_tf (Query corpus) * IDF (Doc corpus)
            tf_idfTerm = raw_tf * termIDF
            tf_idfQueries[queryID][term] = tf_idfTerm

print('First query TF-IDF:')
print (tf_idfQueries['1'])

First query TF-IDF:
{'crystallin': 2.169002281505364, 'len': 1.401316464799885, 'vertebr': 2.412040330191658, 'includ': 1.1390390581279206, 'human': 0.957195470183148}


#### Create a document-term scores dictionary --- WITH RESPECT TO A SINGLE DOCUMENT.

In [6]:
#Dictionary storing accumulated TF-IDF product scores.
#{query ID as key | nested dictionary docID_termScore as value}.
#Term ranking score: tf_idf of the term in query * its raw term frequency with respect to doc ID * its idf in doc corpus.
doc_termScores = {}

#Looking at each query term.
for queryID, queryTerms in tf_idfQueries.items():
    doc_termScores[queryID] = {}
    for term, tf_idfQuery in queryTerms.items():
        
        #Retrieve term if available in doc corpus vocab.
        if term in term_invIndex:
            #Get idf in doc corpus and positing list.
            termIDF_corpus = term_invIndex[term][0]
            postingList = term_invIndex[term][1]
            
            #Get raw term frequency with respect to doc ID. 
            for docID, docTF in postingList.items():
                
                #If doc was not previously retrieved.
                if docID not in doc_termScores[queryID]:
                    doc_termScores[queryID][docID] = 0.0
                    
                doc_termScores[queryID][docID] += tf_idfQuery * docTF * termIDF_corpus

### Step 4: Compute Cosine

#### Compute the length of the vector for the queries. -- Logic same as doc length.

In [7]:
#Dictionary {query ID as key | vector length of it as value}.
query_lenDict = {} 

#Iterate over each query in tf_idfQueries
for queryID, termScores in tf_idfQueries.items(): 
    
    #Accumulator for each individual Sq of the TF-IDF value within a query.
    query_tf_idfSq = 0.0

    #Iterate over each term in the query.
    for term, tf_idfTerm in termScores.items():
        query_tf_idfSq += math.pow(tf_idfTerm, 2.0)

    #Take square root of of the accumulated Sq of term tf_idf
    query_lenDict[queryID] = math.sqrt(query_tf_idfSq)
    
print('Length of the vector for each query:')
print(query_lenDict)

Length of the vector for each query:
{'1': 3.8340357888582473, '2': 5.207790326731164, '3': 3.9297503247686905, '4': 3.5011867079842705, '5': 6.0027593875281084, '6': 3.6819295242023125, '7': 8.973309097542918, '8': 4.891318891084447, '9': 5.428657603192704, '10': 2.5098516927675893, '11': 4.128791494677564, '12': 4.797046938429461, '13': 4.778762493538398, '14': 7.307060337836872, '15': 6.344771177441073, '16': 5.9451508345390875, '17': 6.059060370294427, '18': 3.2088899359441747, '19': 3.3651122913770832, '20': 10.349813675699584, '21': 2.8471593353732514, '22': 6.170576471189581, '23': 2.3520320860722372, '24': 6.519564352044149, '25': 7.119969548504723, '26': 3.388234739891549, '27': 15.425305388401375, '28': 4.424085006061352, '29': 11.327630105367925, '30': 6.006551795231742}


#### Compute Cosine similarity

In [8]:
#Dictionary {query ID as key | nested dictionary as value}.
#Nested dictionary: {doc ID as keys | cosine similarity as values}.
cosineSim_scoreDict = {}

#Each query, get vector length. 
for queryID in doc_termScores.keys():
    queryLength = query_lenDict.get(queryID, 0.0)
    cosineSim_scoreDict[queryID] = {}
    
    #Each doc, get vector length.
    for docID, doc_termScore in doc_termScores[queryID].items():
        docLength = docID_lenDict.get(docID, 0.0)
        
        #Check if lengths are greater than 0, denominator cannot be zero.
        if queryLength > 0 and docLength > 0:
            
            #Cosine similarity score.
            cosineSim = doc_termScore / (queryLength * docLength)
            cosineSim_scoreDict[queryID][docID] = cosineSim
        else:
            cosineSim_scoreDict[queryID][docID] = 0.0

### Step 5: Sort scores in descending order and write output

#### Sort scores

In [9]:
for queryID, docScores in cosineSim_scoreDict.items():
    
    sorted_docScores = sorted(docScores.items(), key=lambda x: x[1], reverse=True)
    cosineSim_scoreDict[queryID] = dict(sorted_docScores)

#Showing results like what you posted in the discussion. 
print(f"Query 1 [#doc={len(cosineSim_scoreDict['1'])}] - FIRST 17 matched documents: \n")
print(f"{dict(islice(cosineSim_scoreDict['1'].items(),17))}")

Query 1 [#doc=223] - FIRST 17 matched documents: 

{'72': 0.3661556461202483, '500': 0.3159256249993626, '965': 0.26138210670241724, '360': 0.20695320831826586, '171': 0.15505750411543895, '15': 0.15257626868366772, '166': 0.15165416970242887, '181': 0.14376422271927988, '513': 0.14197769739495128, '511': 0.12042775273226675, '184': 0.11523683306765448, '13': 0.11434807620744149, '167': 0.11398234525209128, '212': 0.11004970616680217, '138': 0.10915797336220515, '79': 0.10321366011991648, '509': 0.09994691727676606}


#### Re-organized ranked score. Excluding term scores.

In [10]:
#For each query ID, list of ranked docs. 
docRank = {}

for queryID, docScores in cosineSim_scoreDict.items():
    docIDs = list(docScores.keys())
    docRank[queryID] = docIDs

#### Write output

In [11]:
with open ('rankedlist.txt', 'w') as outFile:
    for queryID, docIDs in docRank.items():
        #Print template.
        docIDs_template = ', '.join(docIDs)
        outFile.write(f"{queryID}, {docIDs_template}\n")

print('Successfully written!')

Successfully written!


### Evaluation metric Mean Average Precision -- Compute MAP

#### Open both files -- USE DICTIONARY due to different row length. 

In [12]:
def readFile(fileName):
    '''Read both files as dicitonaries {query ID as key | doc ID as value}.
    Seperated by white space for medline.rel; by (', ') for rankedlist.txt.'''
    
    dataDict = {}
    if fileName == 'medline.rel':
        with open(fileName, 'r') as inFile:
            for line in inFile:
                parts = line.split()
                queryID = parts[0]
                docIDs = list(parts[1:])
                dataDict[queryID] = docIDs
    else: 
        with open(fileName, 'r') as file:
            for line in file:
                parts = line.split(', ')
                queryID = parts[0]
                docIDs = [docID.strip() for docID in parts[1:]]  
                dataDict[queryID] = docIDs
    
    return dataDict

In [13]:
evalDict = readFile('medline.rel')

print('1st query - medLine.rel:')
print(f"{evalDict['1']}")

1st query - medLine.rel:
['13', '14', '15', '72', '79', '138', '142', '164', '165', '166', '167', '168', '169', '170', '171', '172', '180', '181', '182', '183', '184', '185', '186', '211', '212', '499', '500', '501', '502', '503', '504', '506', '507', '508', '510', '511', '513']


In [14]:
rankedDict = readFile('rankedlist.txt')

print('1st query - rankedlist.txt:')
print(f"{rankedDict['1']}")

1st query - rankedlist.txt:
['72', '500', '965', '360', '171', '15', '166', '181', '513', '511', '184', '13', '167', '212', '138', '79', '509', '182', '512', '168', '170', '185', '164', '169', '172', '838', '175', '142', '14', '336', '186', '637', '180', '499', '986', '211', '645', '165', '183', '403', '401', '727', '506', '58', '177', '899', '549', '763', '256', '65', '570', '504', '11', '873', '206', '173', '231', '869', '99', '510', '125', '540', '501', '178', '127', '876', '640', '896', '542', '870', '404', '9', '209', '578', '758', '900', '913', '503', '654', '41', '174', '215', '112', '464', '213', '603', '816', '445', '642', '156', '163', '507', '383', '685', '333', '590', '981', '262', '660', '541', '584', '130', '259', '950', '744', '442', '469', '220', '875', '253', '508', '658', '835', '648', '606', '374', '700', '372', '589', '234', '516', '730', '567', '734', '160', '78', '643', '86', '248', '505', '619', '593', '849', '454', '342', '650', '826', '82', '87', '880', '375', 

#### Average Precision for each query.

In [15]:
def avgPrecision_perQuery (rankedDict, evalDict):
    '''Inputs two dictionaries: computed rank, and relevance for evaluation.'''
    '''Return average precision score for each query.'''
    
    #Dictionary {query ID as key | respective average precision as value}.
    avgPrecision_dict = {}
    
    for queryID, docID_list in rankedDict.items():
            precisionK = 0.0
            relevantCount = 0
            for k, docID in enumerate(docID_list, start=1):
                if docID in evalDict[queryID]:
                    
                    relevantCount += 1
                    precisionK += relevantCount / k
            
            avgPrecision = precisionK / relevantCount
            #print(queryID, avgPrecision)
            avgPrecision_dict[queryID] = avgPrecision
    return avgPrecision_dict

In [16]:
avgPrecision_dict = avgPrecision_perQuery(rankedDict, evalDict)
print('Average precision for each query:')
avgPrecision_dict

Average precision for each query:


{'1': 0.7293854808442106,
 '2': 0.4289898694562347,
 '3': 0.5870782588251162,
 '4': 0.3185516515673878,
 '5': 0.774593708518988,
 '6': 0.7319987566685877,
 '7': 0.6193916474616104,
 '8': 0.45981430849851906,
 '9': 0.48646588164225285,
 '10': 0.575987465693348,
 '11': 0.4864665479828646,
 '12': 0.5759222949615418,
 '13': 0.9182293755070582,
 '14': 0.6324086048535709,
 '15': 0.5462439920811397,
 '16': 0.5810905131283283,
 '17': 0.2988271045816611,
 '18': 0.474891720090341,
 '19': 0.484011394055033,
 '20': 0.2203353933993955,
 '21': 0.3556434832069594,
 '22': 0.27928007125783366,
 '23': 0.8676803142707383,
 '24': 0.768025067969173,
 '25': 0.7244897511812423,
 '26': 0.5337814635400168,
 '27': 0.5549464206513848,
 '28': 0.5420939191163882,
 '29': 0.8027244937762165,
 '30': 0.6104667011732228}

#### Mean Average Precision

In [17]:
mapScore = sum(avgPre_query for avgPre_query in avgPrecision_dict.values())/len(avgPrecision_dict)
print(f'MAP score: {mapScore}')

MAP score: 0.5656605218653455
